In [241]:
import numpy as np
import open3d as o3d
import pickle
from os import path
import matplotlib.pyplot as plt
import time


In [242]:
import sys 
from os import path
sys.path.append(path.relpath('../../.'))
sys.path.append(path.relpath('../../CenterPoint'))
from CenterPoint.tools.visual import center_to_corner_box3d, label2color, corners_to_lines

In [243]:
with open("../../results/detection_pred.pkl", 'rb') as f:
    detection_preds_dict = pickle.load(f)

In [244]:
#frame_name = "39847154216997509_6440_000_6460_000-1568954807924818"
#frame_name = "6503078254504013503_3440_000_3460_000-1557855930472609"
frame_name = "7240042450405902042_580_000_600_000-1559312894537967"
#frame_name = "792520390268391604_780_000_800_000-1557276788222160"
#frame_name = "2257381802419655779_820_000_840_000-1558402112447836"

In [245]:
pred_detections = detection_preds_dict[frame_name]

In [246]:
pc_name = path.join("../../results/lidar", frame_name +".pkl")
with open(pc_name, 'rb') as f:
    lidar_dict = pickle.load(f)

In [247]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(lidar_dict["lidars"]["points_xyz"])
pcd.paint_uniform_color([0.8,0.8,0.8])

PointCloud with 188154 points.

In [248]:
pred_detections["box3d_lidar"]

array([[-1.6305780e+01, -2.9985735e-01,  7.0989197e-01, ...,
         0.0000000e+00,  0.0000000e+00,  1.7578602e-03],
       [ 2.2708923e+01, -9.4064674e+00,  1.2150177e+00, ...,
         0.0000000e+00,  0.0000000e+00, -2.1505718e+00],
       [ 6.1421669e+01,  1.3807109e+01,  9.8410922e-01, ...,
         0.0000000e+00,  0.0000000e+00,  4.1654348e-02],
       ...,
       [ 2.0077808e+01, -1.0403754e+01,  1.0939202e+00, ...,
         0.0000000e+00,  0.0000000e+00, -3.4553933e+00],
       [-1.6868788e+01, -3.6774914e+01,  1.4971629e+00, ...,
         0.0000000e+00,  0.0000000e+00, -1.2188206e+00],
       [ 8.1310921e+00, -5.4835213e+01,  2.5890925e+00, ...,
         0.0000000e+00,  0.0000000e+00, -3.9777141e+00]], dtype=float32)

In [249]:
cluster_name = path.join("../../results/cluster", frame_name +".pkl")
with open(cluster_name, 'rb') as f:
    cluster_dict = pickle.load(f) #x,y,z,cluster,label
labeled_clusters = cluster_dict["cluster_pcs"]

In [250]:
clusters = np.delete(labeled_clusters, -1, axis=1)

## Visualize clustering

In [251]:
sorted_clusters = clusters[clusters[:,-1].argsort()]
np.unique(sorted_clusters[:,-1])
cluster_list = np.split(sorted_clusters[:,:-1], np.unique(sorted_clusters[:,-1], return_index=True)[1][1:])

In [252]:
cluster_bboxes = [o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster)) for cluster in cluster_list]

In [253]:
cluster_line_sets = [o3d.geometry.LineSet().create_from_axis_aligned_bounding_box(box) for box in cluster_bboxes]
for lineset in cluster_line_sets:
    lineset.colors = o3d.utility.Vector3dVector([[255, 0,0] for i in range(12)])

In [254]:
clusters_pc = o3d.geometry.PointCloud()
clusters_pc.points = o3d.utility.Vector3dVector(clusters[:,:-1])
cluster_labels = clusters[:,-1]
max_label = cluster_labels.max()
print(f"point cloud has {int(max_label + 1)} clusters")
colors = plt.get_cmap("tab20b")(cluster_labels / (max_label if max_label > 0 else 1))
colors[cluster_labels < 0] = 0
clusters_pc.colors = o3d.utility.Vector3dVector(colors[:, :3])

point cloud has 3 clusters


In [255]:
SCORE_THRESHOLD = 0.5

In [256]:
def plot_boxes(predictions, score_thresh):
    visuals =[] 
    num_det = predictions['scores'].shape[0]
    for i in range(num_det):
        score = predictions['scores'][i]
        if score < score_thresh:
            continue 

        box = predictions['box3d_lidar'][i:i+1]#.numpy()
        label = predictions['label_preds'][i]
        corner = center_to_corner_box3d(box[:, :3], box[:, 3:6], box[:, -1])[0].tolist()
        color = label2color(int(label) -1)
        visuals.append(corners_to_lines(corner, color))
    return visuals

In [257]:
visuals = [pcd, clusters_pc]
visuals += cluster_line_sets
pred_box_linesets = plot_boxes(pred_detections, SCORE_THRESHOLD)
for lineset in pred_box_linesets:
    lineset.colors = o3d.utility.Vector3dVector([[0, 0,0] for i in range(12)])
#visuals += pred_box_linesets
o3d.visualization.draw_geometries(visuals, width=3072, height=1920)

In [258]:
def center_to_AABB_box3d(boxes):
    AABBs =[] 
    num_det = len(boxes)
    for i in range(num_det):
        box = boxes[i:i+1]
        corner = np.array(center_to_corner_box3d(box[:, :3], box[:, 3:6], box[:, -1])[0].tolist())
        AABBs.append(o3d.geometry.AxisAlignedBoundingBox(corner.min(axis=0), corner.max(axis=0)))
    return AABBs

In [259]:
pred_detections["box3d_lidar"].shape

(166, 9)

In [260]:
pred_bboxes_points = pred_detections["box3d_lidar"][np.argwhere(pred_detections["scores"] > SCORE_THRESHOLD).flatten()]

In [261]:
AABBs = center_to_AABB_box3d(pred_bboxes_points)

In [262]:
PIBB_THRESHOLD = 0.5

In [263]:
for lineset in pred_box_linesets:
    lineset.colors = o3d.utility.Vector3dVector([[0, 0, 0] for i in range(12)])

In [275]:
start = time.time()
cluster_pc_list = []
for cluster_index, cluster_points in zip(range(len(cluster_list)), cluster_list):    
    cluster_pc = o3d.geometry.PointCloud()
    cluster_pc.points = o3d.utility.Vector3dVector(cluster_points)
    cluster_pc.paint_uniform_color([1,0,0])
    cluster_pc_list.append(cluster_pc)
    for AABB_index, AABB in zip(range(len(AABBs)), AABBs):
        num_points = len(cluster_points)
        points_in_AABB_index = AABB.get_point_indices_within_bounding_box(o3d.utility.Vector3dVector(cluster_points))
        if((len(points_in_AABB_index) / num_points) >= PIBB_THRESHOLD):
            colors = np.asarray(cluster_pc.colors)
            colors[points_in_AABB_index] = [0, 255, 0]
            cluster_pc_list[-1].colors = o3d.utility.Vector3dVector(colors)
            
            pred_box_linesets[AABB_index].colors = o3d.utility.Vector3dVector([[0, 255, 0] for i in range(12)])
            cluster_line_sets[cluster_index].colors = o3d.utility.Vector3dVector([[0, 255, 0] for i in range(12)])
            break


end = time.time()
print("Took {} seconds".format(end - start))

Took 0.03980517387390137 seconds


In [276]:
visuals = [pcd]
visuals += cluster_line_sets
visuals += pred_box_linesets
visuals += cluster_pc_list
o3d.visualization.draw_geometries(visuals, width=3072, height=1920,)